In [783]:
import gymnasium as gym
import numpy as np
import scipy.linalg as la
from scipy.linalg import expm
from scipy.stats import unitary_group
from scipy.spatial.transform import Rotation as R
from sympy import Matrix
from typing import Tuple

import cmath
import random
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from qutip.superoperator import liouvillian, spre, spost
from qutip import Qobj, tensor
from qutip.operators import *
from qutip import cnot, cphase

from qiskit.quantum_info import OneQubitEulerDecomposer

sig_p = np.array([[0, 1.], [0, 0]])
sig_m = np.array([[0, 0], [1., 0]])
X = np.array([[0, 1.], [1., 0]])
Z = np.array([[1., 0], [0, -1.]])
I = np.array([[1., 0], [0, 1.]])
Y = np.array([[0, -1.j], [1.j, 0]])

H = np.array([[1/np.sqrt(2),1/np.sqrt(2)],[1/np.sqrt(2),-1/np.sqrt(2)]])
S = np.array([[1.,0],[0,1.j]])
Sdagger = np.array([[1.,0],[0,-1.j]])

#two-qubit single qubit gates
II = tensor(Qobj(I),Qobj(I)).data.toarray()
X1 = tensor(Qobj(X),Qobj(I)).data.toarray()
X2 = tensor(Qobj(I),Qobj(X)).data.toarray()
Y1 = tensor(Qobj(Y),Qobj(I)).data.toarray()
Y2 = tensor(Qobj(I),Qobj(Y)).data.toarray()
Z1 = tensor(Qobj(Z),Qobj(I)).data.toarray()
Z2 = tensor(Qobj(I),Qobj(Z)).data.toarray()

sig_p1 = tensor(Qobj(sig_p),Qobj(I)).data.toarray()
sig_p2 = tensor(Qobj(I),Qobj(sig_p)).data.toarray()
sig_m1 = tensor(Qobj(sig_m),Qobj(I)).data.toarray()
sig_m2 = tensor(Qobj(I),Qobj(sig_m)).data.toarray()
sigmap1 = Qobj(sig_p1)
sigmap2 = Qobj(sig_p2)
sigmam1 = Qobj(sig_m1)
sigmam2 = Qobj(sig_m2)

#two-qubit gate basis
XX = tensor(Qobj(X),Qobj(X)).data.toarray()
YY = tensor(Qobj(Y),Qobj(Y)).data.toarray()
ZZ = tensor(Qobj(Z),Qobj(Z)).data.toarray()
exchangeOperator1 = XX+YY
exchangeOperator2 = YY+ZZ
exchangeOperator3 = XX+ZZ


In [784]:
def decompose_one_qubit_product(
    U: np.ndarray, validate_input: bool = True, atol: float = 1e-8, rtol: float = 1e-5
):
    """
    Decompose a 4x4 unitary matrix to two 2x2 unitary matrices.
    Args:
        U (np.ndarray): input 4x4 unitary matrix to decompose.
        validate_input (bool): if check input.
    Returns:
        phase (float): global phase.
        U1 (np.ndarray): decomposed unitary matrix U1.
        U2 (np.ndarray): decomposed unitary matrix U2.
        atol (float): absolute tolerance of loss.
        rtol (float): relative tolerance of loss.
    Raises:
        AssertionError: if the input is not a 4x4 unitary or
        cannot be decomposed.
    """

    """if validate_input:
        assert np.allclose(
            makhlin_invariants(U, atol=atol, rtol=rtol), (1, 0, 3), atol=atol, rtol=rtol
        )"""

    i, j = np.unravel_index(np.argmax(U, axis=None), U.shape)

    def u1_set(i):
        return (1, 3) if i % 2 else (0, 2)

    def u2_set(i):
        return (0, 1) if i < 2 else (2, 3)

    u1 = U[np.ix_(u1_set(i), u1_set(j))]
    u2 = U[np.ix_(u2_set(i), u2_set(j))]
    
    u1 = to_su(u1)
    u2 = to_su(u2)

    phase = U[i, j] / (u1[i // 2, j // 2] * u2[i % 2, j % 2])

    return phase, u1, u2

def to_su(u: np.ndarray) -> np.ndarray:
    """
    Given a unitary in U(N), return the
    unitary in SU(N).
    Args:
        u (np.ndarray): The unitary in U(N).
    Returns:
        np.ndarray: The unitary in SU(N)
    """

    return u * complex(np.linalg.det(u)) ** (-1 / np.shape(u)[0])

def KAK_2q(
    U: np.ndarray,
    rounding: int = 19
) -> Tuple[float, np.ndarray, np.ndarray, float, np.ndarray, np.ndarray, float,
           float, float]:
    """
    Decomposes a 2-qubit unitary matrix into the product of three matrices:
    KAK = L @ CAN(theta_vec) @ R where L and R are two-qubit local unitaries, 
    CAN is a 3-parameter canonical matrix, and theta_vec is a vector of 3 angles.

    Args:
        U (np.ndarray): 2-qubit unitary matrix
        rounding (int): Number of decimal places to round intermediate 
        matrices to (default 14)

    Returns:
        Tuple of 9 values:
            - phase1 (float): Global phase factor for left local unitary L
            - L1 (np.ndarray): Top 2x2 matrix of left local unitary L
            - L2 (np.ndarray): Bottom 2x2 matrix of left local unitary L
            - phase2 (float): Global phase factor for right local unitary R
            - R1 (np.ndarray): Top 2x2 matrix of right local unitary R
            - R2 (np.ndarray): Bottom 2x2 matrix of right local unitary R
            - c0 (float): XX canonical parameter in the Weyl chamber
            - c1 (float): YY canonical parameter in the Weyl chamber
            - c2 (float): ZZ canonical parameter in the Weyl chamber
    """

    # 0. Map U(4) to SU(4) (and phase)
    U = U / np.linalg.det(U)**0.25 

    assert np.isclose(np.linalg.det(U), 1), "Determinant of U is not 1"

    # 1. Unconjugate U into the magic basis
    B = 1 / np.sqrt(2) * np.array([[1., 0, 0, 1.j], [0, 1.j, 1., 0],
                                   [0, 1.j, -1., 0], [1., 0, 0, -1.j]]) # Magic Basis
    
    U_prime = np.conj(B).T @ U @ B

    # Isolating the maximal torus
    Theta = lambda U: np.conj(U)
    M_squared = Theta(np.conj(U_prime).T) @ U_prime
    
    if rounding is not None:
        M_squared = np.round(M_squared, rounding)  # For numerical stability

    ## 2. Diagonalizing M^2
    D, P = np.linalg.eig(M_squared)
    
    ## Check and correct for det(P) = -1
    if np.isclose(np.linalg.det(P), -1):
        P[:, 0] *= -1  # Multiply the first eigenvector by -1

    # 3. Extracting K2
    K2 = np.conj(P).T

    assert np.allclose(K2 @ K2.T, np.identity(4)), "K2 is not orthogonal"
    assert np.isclose(np.linalg.det(K2), 1), "Determinant of K2 is not 1"

    # 4. Extracting A
    A = np.sqrt(D)
    
    ## Check and correct for det(A) = -1
    if np.isclose(np.prod(A), -1):
        A[0] *= -1  # Multiply the first eigenvalue by -1

    A = np.diag(A)  # Turn the list of eigenvalues into a diagonal matrix
    
    assert np.isclose(np.linalg.det(A), 1), "Determinant of A is not 1"
    
    # 5. Extracting K1
    K1 = U_prime @ np.conj(K2).T @ np.conj(A).T
    
    assert np.allclose(K1 @ K1.T, np.identity(4)), "K1 is not orthogonal"
    assert np.isclose(np.linalg.det(K1), 1), "Determinant of K1 is not 1"

    # 6. Extracting Local Gates
    L = B @ K1 @ np.conj(B).T  # Left Local Product
    R = B @ K2 @ np.conj(B).T  # Right Local Product
    
    phase1, L1, L2 = decompose_one_qubit_product(L)  # L1 (top), L2(bottom)
    phase2, R1, R2 = decompose_one_qubit_product(R)  # R1 (top), R2(bottom)

    # 7. Extracting the Canonical Parameters
    C = np.array([[1, 1, 1], [-1, 1, -1], [1, -1, -1]])  # Coefficient Matrix

    theta_vec = np.angle(np.diag(A))[:3]  # theta vector
    a0, a1, a2 = np.linalg.inv(C) @ theta_vec  # Computing the "a"-vector

    # 8. Unpack Parameters and Put into Weyl chamber
    c0, c1, c2 = 2*a1, -2*a0, 2*a2 # Unpack parameters
    
    assert np.allclose(U, (phase1 * np.kron(L1, L2)) @ CAN(c0, c1, c2)
                       @ (phase2 * np.kron(R1, R2)), atol=1e-03), "U does not equal KAK"

    # np.set_printoptions(suppress=True)
    # print(U)
    # print((phase1 * np.kron(L1, L2)) @ CAN(c0, c1, c2) @ (phase2 * np.kron(R1, R2)))
    
    return phase1, L1, L2, phase2, R1, R2, c0, c1, c2

CAN = lambda c0, c1, c2: expm(1j/2*(c0*np.kron(X, X) + c1*np.kron(Y, Y) + c2*np.kron(Z, Z)))

In [785]:
def singleQubitActionCalculation(U):
        
    singleQubitActions = np.zeros(3)
    
    x_angle , z_angle_after, z_angle_before = OneQubitEulerDecomposer(basis='ZXZ').angles(U)

    singleQubitActions[0] = np.mod((z_angle_after + z_angle_before) / np.pi, 2)  ## alpha
    singleQubitActions[1] = np.mod(x_angle / np.pi,2) ## gamma_magnitude
    singleQubitActions[2] = np.mod(- z_angle_before / np.pi,2)  ## gamma_phase
    
    return singleQubitActions
    
def Rn(theta, axisSelection):
    return np.cos(theta/2)*I-1j*np.sin(theta/2)*axisSelection

def ZXZ_Rotation_Generation( angles):
    return unitary_normalization(Rn(angles[1],Z)@Rn(angles[0],X)@Rn(angles[2],Z))

def unitary_normalization(unitary_in):
    if unitary_in[0][0]==0:
        unitary_in = unitary_in/unitary_in[0][1]
    else:
        unitary_in = unitary_in/unitary_in[0][0]
        
    return unitary_in

def canonicalActionCalculation( c0, c1, c2, index=1):
    
    twoQubitAction = 0
    
    if index == 1:
        b = 1/2*(c0+c1-c2)
    elif index ==2:
        b = 1/2*(-c0+c1+c2)
    elif index ==3:
        b = 1/2*(c0-c1+c2)
    else:
        print("wrong input index")
        
    twoQubitAction = - b / np.pi

    return twoQubitAction

def hamiltonian(delta1, delta2, alpha1, alpha2, g_eff, gamma_magnitude1, gamma_phase1, gamma_magnitude2, gamma_phase2, index = 1):
    selfEnergyTerms = (delta1 + alpha1) * Z1 + (delta2 + alpha2) * Z2
    Qubit1ControlTerms = gamma_magnitude1 * (np.cos(gamma_phase1) * X1 + np.sin(gamma_phase1) * Y1)
    Qubit2ControlTerms = gamma_magnitude2 * (np.cos(gamma_phase2) * X2 + np.sin(gamma_phase2) * Y2)
    
    if index ==1:
        interactionEnergy = g_eff*exchangeOperator1
    elif index ==2:
        interactionEnergy = g_eff*exchangeOperator2
    elif index ==3:
        interactionEnergy = g_eff*exchangeOperator3
    else:
        interactionEnergy = 0
        print("interaction kind not specified")

    energyTotal = selfEnergyTerms + interactionEnergy + Qubit1ControlTerms + Qubit2ControlTerms

    return energyTotal

def hamiltonianSingle(delta1, alpha1, gamma_magnitude1, gamma_phase1):
    selfEnergyTerms = (delta1 + alpha1) * Z
    Qubit1ControlTerms = gamma_magnitude1 * (np.cos(gamma_phase1) * X + np.sin(gamma_phase1) * Y)

    energyTotal = selfEnergyTerms + Qubit1ControlTerms
    
    return energyTotal

def KakActionCalculation(U):
        
    phase1, L1, L2, phase2, R1, R2, c0, c1, c2 = KAK_2q(U)
    
    initialActions = np.zeros(27)
    
    initialActions[0] = singleQubitActionCalculation(R1)[0]
    initialActions[1] = singleQubitActionCalculation(R2)[0]
    initialActions[2] = singleQubitActionCalculation(R1)[1]
    initialActions[3] = singleQubitActionCalculation(R2)[1]
    initialActions[4] = singleQubitActionCalculation(R1)[2]
    initialActions[5] = singleQubitActionCalculation(R2)[2]
    
    initialActions[6] = canonicalActionCalculation(c0,c1,c2,1)
    
    initialActions[7]  = singleQubitActionCalculation(H)[0]
    initialActions[8]  = singleQubitActionCalculation(H)[0]
    initialActions[9]  = singleQubitActionCalculation(H)[1]
    initialActions[10]  = singleQubitActionCalculation(H)[1]
    initialActions[11]  = singleQubitActionCalculation(H)[2]
    initialActions[12]  = singleQubitActionCalculation(H)[2]
    
    initialActions[13] = canonicalActionCalculation(c0,c1,c2,2)
    
    initialActions[14]  = singleQubitActionCalculation(S)[0]
    initialActions[15]  = singleQubitActionCalculation(S)[0]
    initialActions[16]  = singleQubitActionCalculation(S)[1]
    initialActions[17]  = singleQubitActionCalculation(S)[1]
    initialActions[18]  = singleQubitActionCalculation(S)[2]
    initialActions[19]  = singleQubitActionCalculation(S)[2]
    
    initialActions[20] = canonicalActionCalculation(c0,c1,c2,3)
    
    initialActions[21] = singleQubitActionCalculation(L1@H@Sdagger)[0]
    initialActions[22] = singleQubitActionCalculation(L2@H@Sdagger)[0]
    initialActions[23] = singleQubitActionCalculation(L1@H@Sdagger)[1]
    initialActions[24] = singleQubitActionCalculation(L2@H@Sdagger)[1]
    initialActions[25] = singleQubitActionCalculation(L1@H@Sdagger)[2]
    initialActions[26] = singleQubitActionCalculation(L2@H@Sdagger)[2]

    # initialActions[21] = singleQubitActionCalculation(L1)[0]
    # initialActions[22] = singleQubitActionCalculation(L2)[0]
    # initialActions[23] = singleQubitActionCalculation(L1)[1]
    # initialActions[24] = singleQubitActionCalculation(L2)[1]
    # initialActions[25] = singleQubitActionCalculation(L1)[2]
    # initialActions[26] = singleQubitActionCalculation(L2)[2]
        
    return initialActions
    

In [786]:
def PulseGen(Actions):
    
    final_time = 30E-9 # in seconds, total time is final_time * 5 because of single qubit + two_qubit + single_qubit + two_qubit + single_qubit
    num_Haar_basis = 2  # number of Haar basis (need to update for odd combinations)
    steps_per_Haar = 1  # steps per Haar basis per episode
    
    PiFreq = np.pi / final_time
    current_Haar_num = 1  # starting with 1
    current_step_per_Haar = 1
    alpha_max = PiFreq / 2
    g_eff_max = PiFreq / 2
    gamma_phase_max = np.pi
    gamma_magnitude_max = PiFreq / 2
    
    num_time_bins = 2

    H2_1_array = []  # Array of Hs at each Haar wavelet
    H2_2_array = []  # Array of Hs at each Haar wavelet
    H2_3_array = []  # Array of Hs at each Haar wavelet

    H1_1_array = []  # Array of Hs at each Haar wavelet
    H1_2_array = []  # Array of Hs at each Haar wavelet
    H1_3_array = []  # Array of Hs at each Haar wavelet
    H1_4_array = []  # Array of Hs at each Haar wavelet

    
    for ii in range(num_Haar_basis):

        if current_Haar_num==1:
            alpha1_1 = alpha_max * (Actions[0])
            alpha2_1 = alpha_max * (Actions[1])

            gamma_magnitude1_1 = gamma_magnitude_max * (Actions[2])
            gamma_magnitude2_1 = gamma_magnitude_max * (Actions[3])

            gamma_phase1_1 = gamma_phase_max * (Actions[4])
            gamma_phase2_1 = gamma_phase_max * (Actions[5])
        elif current_Haar_num==2:
            alpha1_1 = - alpha_max * (Actions[0])
            alpha2_1 = - alpha_max * (Actions[1])

            gamma_magnitude1_1 = gamma_magnitude_max * (Actions[2])
            gamma_magnitude2_1 = gamma_magnitude_max * (Actions[3])

            gamma_phase1_1 = gamma_phase_max * (Actions[4])
            gamma_phase2_1 = gamma_phase_max * (Actions[5])

        ### First two qubit gate
        
        if current_Haar_num==1:
            g_eff1 = g_eff_max * (Actions[6])

        ### Second Single qubit gate
        if current_Haar_num==1:
            alpha1_2 = alpha_max * ( Actions[7])
            alpha2_2 = alpha_max * ( Actions[8])

            gamma_magnitude1_2 = gamma_magnitude_max * ( Actions[9])
            gamma_magnitude2_2 = gamma_magnitude_max * (Actions[10])

            gamma_phase1_2 = gamma_phase_max * ( Actions[11])
            gamma_phase2_2 = gamma_phase_max * ( Actions[12])
        elif current_Haar_num==2:
            alpha1_2 = - alpha_max * ( Actions[7])
            alpha2_2 = - alpha_max * ( Actions[8])

            gamma_magnitude1_2 = gamma_magnitude_max * ( Actions[9])
            gamma_magnitude2_2 = gamma_magnitude_max * (Actions[10])

            gamma_phase1_2 = gamma_phase_max * ( Actions[11])
            gamma_phase2_2 = gamma_phase_max * ( Actions[12])
            
        ### second two qubit gate
        if current_Haar_num==1:
            g_eff2 = g_eff_max * (Actions[13])

        ### Third Single qubit gate
        if current_Haar_num==1:        
            alpha1_3 = alpha_max * ( Actions[14])
            alpha2_3 = alpha_max * ( Actions[15])

            gamma_magnitude1_3 = gamma_magnitude_max * ( Actions[16])
            gamma_magnitude2_3 = gamma_magnitude_max * ( Actions[17])

            gamma_phase1_3 = gamma_phase_max * ( Actions[18]) 
            gamma_phase2_3 = gamma_phase_max * ( Actions[19])
        elif current_Haar_num==2:
            alpha1_3 = - alpha_max * ( Actions[14])
            alpha2_3 = - alpha_max * ( Actions[15])

            gamma_magnitude1_3 = gamma_magnitude_max * ( Actions[16])
            gamma_magnitude2_3 = gamma_magnitude_max * ( Actions[17])

            gamma_phase1_3 = gamma_phase_max * ( Actions[18]) 
            gamma_phase2_3 = gamma_phase_max * ( Actions[19])

        ### third two qubit gate
        if current_Haar_num==1:
            g_eff3 = g_eff_max * (Actions[20])
        
        ### Fourth Single qubit gate
        if current_Haar_num==1:        
            alpha1_4 = alpha_max * (Actions[21])
            alpha2_4 = alpha_max * (Actions[22])

            gamma_magnitude1_4 = gamma_magnitude_max * (Actions[23])
            gamma_magnitude2_4 = gamma_magnitude_max * (Actions[24])

            gamma_phase1_4 = gamma_phase_max * (Actions[25])
            gamma_phase2_4 = gamma_phase_max * (Actions[26])
        elif current_Haar_num==2:
            alpha1_4 = - alpha_max * (Actions[21])
            alpha2_4 = - alpha_max * (Actions[22])

            gamma_magnitude1_4 = gamma_magnitude_max * ( Actions[23])
            gamma_magnitude2_4 = gamma_magnitude_max * (Actions[24])

            gamma_phase1_4 = gamma_phase_max * (Actions[25])
            gamma_phase2_4 = gamma_phase_max * (Actions[26])


        H2_1 = hamiltonian(0, 0, 0, 0, g_eff1, 0, 0, 0, 0, index = 1)
        H2_2 = hamiltonian(0, 0, 0, 0, g_eff2, 0, 0, 0, 0, index = 1)
        H2_3 = hamiltonian(0, 0, 0, 0, g_eff3, 0, 0, 0, 0, index = 1)
        
        H1_1 = hamiltonian(0, 0, alpha1_1, alpha2_1, 0, gamma_magnitude1_1, gamma_phase1_1, gamma_magnitude2_1, gamma_phase2_1)
        H1_2 = hamiltonian(0, 0, alpha1_2, alpha2_2, 0, gamma_magnitude1_2, gamma_phase1_2, gamma_magnitude2_2, gamma_phase2_2)
        H1_3 = hamiltonian(0, 0, alpha1_3, alpha2_3, 0, gamma_magnitude1_3, gamma_phase1_3, gamma_magnitude2_3, gamma_phase2_3)
        H1_4 = hamiltonian(0, 0, alpha1_4, alpha2_4, 0, gamma_magnitude1_4, gamma_phase1_4, gamma_magnitude2_4, gamma_phase2_4)

        
        H2_1_array.append(H2_1)  # Array of Hs at each Haar wavelet
        H2_2_array.append(H2_2)  # Array of Hs at each Haar wavelet
        H2_3_array.append(H2_3)  # Array of Hs at each Haar wavelet

        H1_1_array.append(H1_1)  # Array of Hs at each Haar wavelet
        H1_2_array.append(H1_2)  # Array of Hs at each Haar wavelet
        H1_3_array.append(H1_3)  # Array of Hs at each Haar wavelet
        H1_4_array.append(H1_4)  # Array of Hs at each Haar wavelet        

        if current_Haar_num == 1:
            current_Haar_num = current_Haar_num + 1

    # H_tot for adding Hs at each time bins
    H_tot2_1 = []
    H_tot2_2 = []
    H_tot2_3 = []

    H_tot1_1 = []
    H_tot1_2 = []
    H_tot1_3 = []
    H_tot1_4 = []
        
    for ii, H_elem in enumerate(H1_1_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot1_1[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot1_1.append(factor * H_elem)          

    for ii, H_elem in enumerate(H2_1_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot2_1[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot2_1.append(factor * H_elem)

    for ii, H_elem in enumerate(H1_2_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot1_2[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot1_2.append(factor * H_elem)

    for ii, H_elem in enumerate(H2_2_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot2_2[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot2_2.append(factor * H_elem)

    for ii, H_elem in enumerate(H1_3_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot1_3[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot1_3.append(factor * H_elem)

    for ii, H_elem in enumerate(H2_3_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot2_3[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot2_3.append(factor * H_elem)

    for ii, H_elem in enumerate(H1_4_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot1_4[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot1_4.append(factor * H_elem)

    unitary_U = np.eye(4)

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot1_1[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot2_1[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot1_2[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot2_2[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot1_3[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot2_3[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot1_4[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U
        
    return unitary_U

def CanonicalPulseGen(Actions, index):
    
    final_time = 30E-9 # in seconds, total time is final_time * 5 because of single qubit + two_qubit + single_qubit + two_qubit + single_qubit
    num_Haar_basis = 2  # number of Haar basis (need to update for odd combinations)
    steps_per_Haar = 1  # steps per Haar basis per episode
    
    PiFreq = np.pi / final_time
    current_Haar_num = 1  # starting with 1
    current_step_per_Haar = 1
    alpha_max = PiFreq / 2
    g_eff_max = PiFreq / 2
    gamma_phase_max = np.pi
    gamma_magnitude_max = PiFreq / 2
    
    num_time_bins = 2

    H2_1_array = []  # Array of Hs at each Haar wavelet
    
    for ii in range(num_Haar_basis):

        if current_Haar_num==1:
            g_eff1 = g_eff_max * (Actions)

        H2_1 = hamiltonian(0, 0, 0, 0, g_eff1, 0, 0, 0, 0, index)

        H2_1_array.append(H2_1)  # Array of Hs at each Haar wavelet

        if current_Haar_num == 1:
            current_Haar_num = current_Haar_num + 1

    # H_tot for adding Hs at each time bins
    H_tot2_1 = []

    for ii, H_elem in enumerate(H2_1_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot2_1[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot2_1.append(factor * H_elem)

    unitary_U = np.eye(4)

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot2_1[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U

    return unitary_U

def SingleQubitPulseGen(Actions):
    
    final_time = 30E-9 # in seconds, total time is final_time * 5 because of single qubit + two_qubit + single_qubit + two_qubit + single_qubit
    num_Haar_basis = 2  # number of Haar basis (need to update for odd combinations)
    steps_per_Haar = 1  # steps per Haar basis per episode
    
    PiFreq = np.pi / final_time
    current_Haar_num = 1  # starting with 1
    current_step_per_Haar = 1
    alpha_max = PiFreq / 2
    gamma_phase_max = np.pi
    gamma_magnitude_max = PiFreq / 2
    
    num_time_bins = 2

    H1_1_array = []  # Array of Hs at each Haar wavelet

    
    for ii in range(num_Haar_basis):

        if current_Haar_num==1:
            alpha1_1 = alpha_max * (Actions[0])

            gamma_magnitude1_1 = gamma_magnitude_max * (Actions[1])

            gamma_phase1_1 = gamma_phase_max * (Actions[2])
        elif current_Haar_num==2:
            alpha1_1 = - alpha_max * (Actions[0])

            gamma_magnitude1_1 = gamma_magnitude_max * (Actions[1])

            gamma_phase1_1 = gamma_phase_max * (Actions[2])

        H1_1 = hamiltonianSingle(0,alpha1_1, gamma_magnitude1_1, gamma_phase1_1)

        
        H1_1_array.append(H1_1)  # Array of Hs at each Haar wavelet

        if current_Haar_num == 1:
            current_Haar_num = current_Haar_num + 1

    H_tot1_1 = []
        
    for ii, H_elem in enumerate(H1_1_array):
        for jj in range(0, num_time_bins):
            Haar_num = current_Haar_num - np.floor(ii / steps_per_Haar) # Haar_num: label which Haar wavelet, current_Haar_num: order in the array
            factor = (-1) ** np.floor(jj / (2 ** (Haar_num - 1))) # factor flips the sign every 2^(Haar_num-1)
            if ii > 0:
                H_tot1_1[jj] += factor * H_elem
            else:  # Because H_tot[jj] does not exist
                H_tot1_1.append(factor * H_elem)          

    unitary_U = np.eye(2)

    for jj in range(0, num_time_bins):
        unitary_Ut = la.expm(-1j * H_tot1_1[jj] * final_time / num_time_bins)
        unitary_U = unitary_Ut @ unitary_U
        
    return unitary_U

In [787]:
U = Rn(0.354*np.pi,X)@S@H@Sdagger
U_synth = SingleQubitPulseGen(singleQubitActionCalculation(U))

In [788]:
U = expm(-1j/2*(-0.7)*np.pi*(np.kron(X,X)+np.kron(Y,Y)))
# U = np.kron(S@H@Rn(0.136*np.pi,Y),Z)
U_synth = PulseGen(KakActionCalculation(U))

In [789]:
phase1, L1, L2, phase2, R1, R2, c0, c1, c2 = KAK_2q(U)

b0pi = canonicalActionCalculation(c0,c1,c2,1)*np.pi
b1pi = canonicalActionCalculation(c0,c1,c2,2)*np.pi
b2pi = canonicalActionCalculation(c0,c1,c2,3)*np.pi

b0 = canonicalActionCalculation(c0,c1,c2,1)
b1 = canonicalActionCalculation(c0,c1,c2,2)
b2 = canonicalActionCalculation(c0,c1,c2,3)

print(unitary_normalization(CAN(c0, c1, c2)))
print(unitary_normalization(CAN(-b0pi, -b0pi, 0) @ CAN(0, -b1pi, -b1pi) @ CAN(-b2pi, 0, -b2pi) ))
print(unitary_normalization(CanonicalPulseGen(b0,1) @ CanonicalPulseGen(b1,2) @ CanonicalPulseGen(b2,3)))
print(unitary_normalization(CanonicalPulseGen(b2,3) @ CanonicalPulseGen(b1,2) @ CanonicalPulseGen(b0,1)))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.50952545j]
 [ 0.        +0.j          0.58778525-0.80901699j -0.41221475-0.29949154j
   0.        +0.j        ]
 [ 0.        +0.j         -0.41221475-0.29949154j  0.58778525-0.80901699j
   0.        +0.j        ]
 [ 0.        +0.50952545j  0.        +0.j          0.        +0.j
   1.        +0.j        ]]
[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.50952545j]
 [ 0.        +0.j          0.58778525-0.80901699j -0.41221475-0.29949154j
   0.        +0.j        ]
 [ 0.        +0.j         -0.41221475-0.29949154j  0.58778525-0.80901699j
   0.        +0.j        ]
 [ 0.        +0.50952545j  0.        +0.j          0.        +0.j
   1.        +0.j        ]]
[[ 1.        +0.j          0.        +0.j          0.        +0.j
  -0.        +0.50952545j]
 [ 0.        +0.j          0.58778525-0.80901699j -0.41221475-0.29949154j
   0.        +0.j        ]
 [ 0.        +0.j         -0

In [790]:
print(CanonicalPulseGen(b0,1) @ CanonicalPulseGen(b1,2) @ CanonicalPulseGen(b2,3))
print(PulseGen(KakActionCalculation(np.kron(H@Sdagger,H@Sdagger))) @ CanonicalPulseGen(b2,1) @ PulseGen(KakActionCalculation(np.kron(S,S))) @ CanonicalPulseGen(b1,1) @ PulseGen(KakActionCalculation(np.kron(H,H))) @ CanonicalPulseGen(b0,1))

[[ 0.79389263+0.4045085j  0.        +0.j         0.        +0.j
  -0.20610737+0.4045085j]
 [ 0.        +0.j         0.79389263-0.4045085j -0.20610737-0.4045085j
   0.        +0.j       ]
 [ 0.        +0.j        -0.20610737-0.4045085j  0.79389263-0.4045085j
   0.        +0.j       ]
 [-0.20610737+0.4045085j  0.        +0.j         0.        +0.j
   0.79389263+0.4045085j]]
[[-0.79389263-0.4045085j -0.        -0.j        -0.        -0.j
  -0.20610737+0.4045085j]
 [ 0.        -0.j        -0.79389263+0.4045085j  0.20610737+0.4045085j
  -0.        -0.j       ]
 [ 0.        -0.j         0.20610737+0.4045085j -0.79389263+0.4045085j
  -0.        -0.j       ]
 [-0.20610737+0.4045085j -0.        -0.j        -0.        -0.j
  -0.79389263-0.4045085j]]


In [791]:
np.set_printoptions(suppress=True)
print((CanonicalPulseGen(1,1)))
print(np.kron(SingleQubitPulseGen(singleQubitActionCalculation(H@Sdagger)),SingleQubitPulseGen(singleQubitActionCalculation(H@Sdagger))) @ CAN(np.pi,np.pi,0) @ np.kron(SingleQubitPulseGen(singleQubitActionCalculation(S@H)),SingleQubitPulseGen(singleQubitActionCalculation(S@H))) )
print((PulseGen(KakActionCalculation(np.kron(H@Sdagger,H@Sdagger))) @ CanonicalPulseGen(1,1) @ PulseGen(KakActionCalculation(np.kron(S@H,S@H)))))

[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j  0.-0.j  0.+0.j]
 [ 0.+0.j  0.-0.j -1.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j]]
[[ 1.-0.j  0.+0.j -0.+0.j  0.-0.j]
 [-0.-0.j -1.-0.j  0.+0.j -0.-0.j]
 [-0.-0.j  0.+0.j -1.-0.j -0.-0.j]
 [ 0.-0.j -0.+0.j  0.+0.j  1.+0.j]]
[[-0.5+0.5j -0. +0.j  -0. -0.j  -0.5-0.5j]
 [-0. +0.j   0.5+0.5j -0.5+0.5j -0. +0.j ]
 [-0. -0.j  -0.5+0.5j  0.5+0.5j -0. +0.j ]
 [-0.5-0.5j  0. -0.j   0. -0.j  -0.5+0.5j]]


In [792]:
print(np.kron(H@Sdagger,H@Sdagger)@CanonicalPulseGen(1,1)@np.kron(S@H,S@H))
print(CanonicalPulseGen(1,3))

[[ 0.-0.j -0.+0.j  0.-0.j  1.+0.j]
 [-0.-0.j  0.+0.j  1.-0.j  0.+0.j]
 [ 0.+0.j  1.-0.j  0.+0.j -0.-0.j]
 [ 1.+0.j  0.-0.j -0.+0.j  0.-0.j]]
[[ 0.-0.j  0.+0.j  0.+0.j -1.-0.j]
 [ 0.+0.j  0.+0.j  1.-0.j  0.+0.j]
 [ 0.+0.j  1.-0.j  0.+0.j  0.+0.j]
 [-1.-0.j  0.+0.j  0.+0.j  0.-0.j]]


In [793]:
print(unitary_normalization(PulseGen(KakActionCalculation(np.kron(S@H,S@H)))))
print(unitary_normalization(np.kron(S@H,S@H)))

[[ 1.+0.j -0.+1.j -0.+1.j  1.+0.j]
 [-0.-1.j  1.-0.j -1.+0.j -0.+1.j]
 [-0.-1.j -1.+0.j  1.-0.j -0.+1.j]
 [-1.+0.j  0.+1.j  0.+1.j -1.+0.j]]
[[ 1.+0.j  1.+0.j  1.+0.j  1.+0.j]
 [ 0.+1.j  0.-1.j  0.+1.j  0.-1.j]
 [ 0.+1.j  0.+1.j  0.-1.j  0.-1.j]
 [-1.+0.j  1.+0.j  1.+0.j -1.+0.j]]


In [794]:
print(unitary_normalization(S@H))
print(unitary_normalization(SingleQubitPulseGen(singleQubitActionCalculation(S@H))))

[[1.+0.j 1.+0.j]
 [0.+1.j 0.-1.j]]
[[ 1.-0.j  1.+0.j]
 [ 0.+1.j -0.-1.j]]


In [795]:
print(np.kron(H@Sdagger,H@Sdagger)@CAN(1,0,0)@np.kron(S@H,S@H))
print(CAN(0,1,0))

[[ 0.87758256+0.j          0.        -0.j         -0.        +0.j
   0.        -0.47942554j]
 [ 0.        +0.j          0.87758256+0.j          0.        +0.47942554j
  -0.        +0.j        ]
 [-0.        +0.j          0.        +0.47942554j  0.87758256+0.j
   0.        +0.j        ]
 [ 0.        -0.47942554j -0.        +0.j          0.        -0.j
   0.87758256+0.j        ]]
[[0.87758256+0.j         0.        +0.j         0.        +0.j
  0.        -0.47942554j]
 [0.        +0.j         0.87758256+0.j         0.        +0.47942554j
  0.        +0.j        ]
 [0.        +0.j         0.        +0.47942554j 0.87758256+0.j
  0.        +0.j        ]
 [0.        -0.47942554j 0.        +0.j         0.        +0.j
  0.87758256+0.j        ]]


In [796]:
np.set_printoptions(suppress=True)
print(unitary_normalization(U))
print(unitary_normalization(U_synth))

[[ 1.        +0.j          0.        +0.j          0.        +0.j
   0.        +0.j        ]
 [ 0.        +0.j         -0.58778525+0.j          0.        +0.80901699j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.80901699j -0.58778525+0.j
   0.        +0.j        ]
 [ 0.        +0.j          0.        +0.j          0.        +0.j
   1.        +0.j        ]]
[[ 1.        -0.j         -0.        -0.j          0.        -0.j
   0.        -0.50952545j]
 [-0.        +0.j         -0.58778525+0.80901699j  0.41221475+0.29949154j
  -0.        +0.j        ]
 [ 0.        +0.j          0.41221475+0.29949154j -0.58778525+0.80901699j
  -0.        +0.j        ]
 [-0.        -0.50952545j -0.        +0.j         -0.        +0.j
   1.        +0.j        ]]
